In [1]:
import re
import math
import sys

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

import pythainlp

from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TescoLotus.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 269
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

appear_rate = 1
dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = []
for word in dictionary.keys():
    if len(dictionary[word]) <= 1:
        letter_words.append(word)
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1313 words
filter frequent words: 540 words
filter letter words: 539 words
filter stop words: 352 words


In [4]:
# bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)
average_doc_size

word_freq = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, word_freq[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### K-Mean

In [6]:
def get_bow(corpus):
    new_dict = Dictionary(corpus)

    # new_dict.filter_extremes(no_below=2, no_above=1, keep_n=len(new_dict))
    # print(len(new_dict))

    unique_words = [new_dict[id] for id in range(len(new_dict))]
    array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)
    
    for i, doc in enumerate(corpus):
        for word in doc:
            array[i, new_dict.token2id[word]] += 1

        ## normalization
        if len(doc) != 0:
            array[i] = numpy.divide(array[i], len(doc))

    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [7]:
def predict_cluster(bow_corpus, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters).fit(bow_corpus)
    predicted_labels = kmeans.labels_

    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    
    return result

In [22]:
num_clusters = 6
# bow_corpus = get_bow(idx_corpus)
bow_corpus = get_bow(filtered_corpus)
result = predict_cluster(bow_corpus, num_clusters)
result.head()

,comment,tokenized_comment,label,predicted_label
0,บางครั้งพนักงานโลตัสเอ็กเพลส กับพนักงานโลตัสให...,"[พนักงาน, พนักงาน, โลตัส, โลตัส, ทำ, เอ็กเพลส,...",,5
1,เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพีย...,"[เลิก, ข้าว, ดี, อี, เครื่อง, น้ำ]",,1
2,ถุงงง ถุงมึงบอบบางมากกก ยิ่งเจอน้ำยาซักผ้า น้...,"[ถุง, งง, ถุง, กก, เจอ, น้ำยา, ซัก, ผ้า, น้ำ, ...",,2
3,เราไม่เคยรับถุงเลย แถมได้แต้มกรีนพ้อยไว้ใช้เป็...,"[ถุง, แถม, ลด, ลอง, ดู, ถุง, หนา]",,2
4,เคยไปทำ เอ้กซ์เพรส เดือนเดียว พอเลย พนักงานต่...,"[ทำ, เดือน, พนักงาน, คน, คน, คน, นึง, พัน, เช้...",,1


### Result

In [23]:
label_count = numpy.unique(result['predicted_label'], return_counts=True) 
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([0, 1, 2, 3, 4, 5], dtype=int32), array([ 16, 201,  24,   4,  10,  14]))
		percent
0  |	16	1.0


In [24]:
comment_widget = widgets.ToggleButtons(
    options=[num for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_comment_widget_click(change):
    clear_output()
    display(comment_widget)
    for index, value in result[result['predicted_label'] == change['new']]['comment'].iteritems():
        print(index, value)

comment_widget.observe(on_comment_widget_click, names='index')
on_comment_widget_click({'new' : 0})

ToggleButtons(index=5, options=(0, 1, 2, 3, 4, 5), value=5)

0 บางครั้งพนักงานโลตัสเอ็กเพลส กับพนักงานโลตัสใหญ่ ก็ต่างกันครับ โลตัสใหญ่ เขาทำเป็นเเผนกๆไปครับ เเต่โลตัสเอ็กเพลสนั้น พนักงานเเต่ละคน ต้องทำทุกอย่างครับ ไม่ใช่เเค่ยืนขายของ อยากให้ทุกคนเข้าใจด้วยครับ บางคนเวลากินข้าวเเทบไม่มี ผมเชื่อว่าทุกคนก็เป็นพนักงานเงินเดือนเหมือนกัน ควรเข้าใจกันบ้างครับ พนักงานคนอื่นเป็นเเบบไหนผมไม่รู้ เเต่มันก็ยังมีพนักงานที่ตั้งใจทำงานครับ ขอให้ทุกคนเข้าใจครับ
39 90 เปอร์เซ็นต์ของคอมเม้นต์นี่ เกี่ยวกับพนักงานล้วนๆ 555รวมทั้งของนี่ด้วย
45 พนักงานชอบเหวี่ยง กระชากเงินจากมือ ยื่นเงินทอนให้แบบไม่เต็มใจ โดยเฉพาะพนักงานพาร์ทไทม์
116 พนักงานขายที่นี่มีการคัดกรองมั้ยคะคุณสมบัติพนักงานที่นี่
184 พนักงานตามเอ็กเพลสนี่ต้องฝึกมารยาทนะ เห็นอยู่กันตามยถากรรม
185 มารยาทพนักงาน
188 ปรับปรุงพนักงานค่ะ
221 พนักงาน
228 มารยาทของพนักงาน
233 เอ๊กเพรส ต้องปรับปรุงพนักงาน
236 พนักงาน
238 พนักงาน
241 มารยาทพนักงานควรปรับปรุงมากที่สุด
253 มารยาทของพนักงานของพนักงานค่ะ


In [25]:
token_widget = widgets.ToggleButtons(
    options=[num for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_token_widget_click(change):
    clear_output()
    display(token_widget)
    for index, value in result[result['predicted_label'] == change['new']]['tokenized_comment'].iteritems():
        print(index, value)

token_widget.observe(on_token_widget_click, names='index')
on_token_widget_click({'new' : 0})

ToggleButtons(options=(0, 1, 2, 3, 4, 5), value=0)

14 ['สาขา', 'กะปิ', 'ตัว', 'หน้า', 'ร้าน', 'ชั้น', 'ชั้น', 'ไปรษณีย์', 'ประตู', 'จอด', 'รถ']
44 ['ช่อง', 'จ่าย', 'เงิน', 'ด่วน', 'สินค้า', 'ชิ้น', 'อย่า', 'เข็น', 'จ่าย', 'เต็ม', 'รถ', 'เข็น', 'พนักงาน', 'สัก', 'ก้อ', 'ห่วย', 'รอ', 'คิว', 'ชาติ', 'ช่อง', 'ช่อง', 'กุ', 'บ้า']
60 ['รถ', 'เข็น', 'เติม', 'สินค้า', 'ลูกค้า', 'ขนาด', 'กกก', 'กลัว', 'ซื้อ']
76 ['ดี', 'คิว', 'ซื้อ', 'พนักงาน', 'ลูกค้า', 'ซื้อ', 'รถ', 'เข็น']
90 ['รถ', 'เข็น', 'หา', 'พัง', 'พนักงาน']
100 ['พนง', 'หน้า', 'แย่', 'ปล่อย', 'เรื่อง', 'จอด', 'รถ', 'ลูกค้า', 'ปกติ', 'จอด', 'คน', 'หน้าตา', 'พนง', 'บัตร', 'จอด', 'รถ', 'คุย', 'เจอ', 'ด่า', 'สาย', 'หยาบ', 'นึก', 'ด่า']
125 ['รถ', 'เข็น', 'เหมือน', 'กิน', 'รถ', 'ปกติ', 'เหมือน', 'หวย']
134 ['สาขา', 'ติด', 'รถ', 'เข็น', 'พื้น', 'หน้า', 'ซื้อ', 'โดน']
151 ['รถ', 'เข็น', 'รถ', 'เด็ก', 'นั่ง', 'สาขา', 'สกปรก']
166 ['ล้อ', 'รถ', 'หลุด', 'พัง', 'คัน']
174 ['รถ', 'เข็น', 'คัน', 'ล้อ', 'ล้อ', 'ซ่อม', 'เข็น', 'ลำบาก', 'งาน']
202 ['จอด', 'รถ', 'ดี', 'ชาย', 'จอด']
208 ['รถ', 'เข็น', 